# Crawler - Psicologos
## Crawler com heurística

Este notebook dispõe de um crawller para páginas contendo instâncias de psicologos, psiquiatras, clínicas e outros profissionais neste domínio.

Este Crawler funcionará como o já apresentado Crawler em largura, porém aplicaremos heurísticas para evitarmos acessar tantas páginas negativas.

In [9]:
import pandas as pd
import numpy as np
import scipy as scp
from bs4 import BeautifulSoup
import requests as rq
import re
import time
import random
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

## Sementes
Partiremos de "sementes", que são sites dos quais já analisamos manualmente anteriormente e é sabido que levam a instâncias de profissionais do nosso domínio.

In [10]:
df = pd.read_csv('sementes.csv')
df

,semente,dominio,robotstxt,localpath,htmlname
0,https://www.srlupa.com.br,srlupa.com.br,False,pages/sementes/srlupa/,srlupa
1,https://serviceira.com,serviceira.com,False,pages/sementes/serviceira/,serviceira
2,https://conectu.com.br,conectu.com.br,False,pages/sementes/conectu/,conectu
3,https://www.achouservicos.com.br,achouservicos.com.br,False,pages/sementes/achouservicos/,achouservicos
4,https://www.telelistas.net/br/clinicas+de+psic...,telelistas.net,True,pages/sementes/telelistas/clinicasdepsicologia/,clinicasdepsicologia
5,https://www.telelistas.net/br/clinicas+psiquia...,telelistas.net,True,pages/sementes/telelistas/clinicaspsiquiatricas/,clinicaspsiquiatricas
6,https://www.telelistas.net/br/psicanalistas,telelistas.net,True,pages/sementes/telelistas/psicanalistas/,psicanalistas
7,https://www.telelistas.net/br/psicologos,telelistas.net,True,pages/sementes/telelistas/psicologos/,psicologos
8,https://www.telelistas.net/br/psicopedagogos,telelistas.net,True,pages/sementes/telelistas/psicopedagogos/,psicopedagogos
9,https://freelancer.com.br,freelancer.com.br,True,pages/sementes/freelancer/,freelancer


## Robots.txt
Nossa primeira preocupação é respeitar o robots.txt disponível no site que vamos crawllear. Há bibliotecas para tal funcionalidade. Para os domínios das sementes já havia, previamente, baixado teus robots.txt (quando existia).

O metodo getDisallows(path) abaixo recebe o path onde está a semente e o robots.txt e, iterando sobre ele, retorna a lista de restrições do robots.txt.

In [11]:
def getDisallows(path):
    ret_list = []
    file = open(path+"robots.txt", "r")
    lines = file.readlines()
    read = False
    for line in lines:
        if(line.endswith("\n")):
            line = line[:-1]
        if(line.lower().startswith("user-agent:") and line.endswith("*")):
            read = True
        elif(line.lower().startswith("user-agent:") and not(line.endswith("*"))):
            read = False
        elif(line.lower().startswith("disallow:") and read):
            aux = line[10:]
            
            if(aux.endswith("*")):
                aux = aux[:-1]
            if(aux.endswith("?")):
                aux = aux[:-1]
            if(aux.endswith("?")):
                aux = aux[:-1]
            if(aux.endswith("*")):
                aux = aux[:-1]
                
            ret_list.append(aux)
    return ret_list

## Crawleando com heurística

Rodo o Crawller partindo de cada semente, iterativamente. É uma busca em heurística, onde - por motivos de eficiencia - só entro numa página uma vez, só acesso links .html e checo se estou dentro dos domínios das sementes (para evitar sair do domínio para um site como, por exemplo, google.com). Também respeitaremos os sites dando um time entre uma requisição e a próxima.

Ainda mais, para evitar termos tantos acessos em páginas negativas, implementaremos algumas heurísticas. São elas:

1 - Não acessar páginas comuns de informações inúteis, como /sobre, /contato, /empresa e etc.

2 - Não salvar páginas com baixa frequencia de bag of words.

3 - Priorizar os links cujo "sublinks" se repetem (uma abstração ao inlink). Por exemplo, domain.com/psico/1 e 
    domain.com/psico/2 compartilham o mesmo prefixo. eles ficam juntos, pois tendem a serem instancias.

4 - Ir treinando um classificador de link periodicamente, a cada 100 iterações.

Iniciando lista de links inúteis para item 1

In [12]:
useless_link_list = ['/sobre','/about','blog.', 'dev.','app.','loja.','/ofereca-seus-servicos',
                     '/ofereca','/accounts','/login','/empresa','/login','/cadastro','/cadastre','/login','/entre',
                    'politica-de-privacidade','/privacy','/termos-e-condicoes-de-uso','/termos-de-uso','/condicoes-de-uso',
                    '/central-de-ajuda','/helpdesk','/help-desk', '/ajuda', '/perguntas-frequentes', '/faq', '/FAQ',
                    '/signup','/service-provider','/terms','/contato','/fale-conosco','/faleconosco','/saibamais','/termos'
                    '/blog','/Aulas','/Assistência+técnica+de+eletrônicos','/DJ','/Mecanico','/Doces','/Eletricista',
                    '/Pedreiro','/Encanador','/Chaveiro','/Fotograf','/Pintor','/Diarista','/Jardinagem','/Veterinário',
                    '/anuncie','/restaurante','/automoveis','/hotel','/promocoes','/parceiros','/cadastramento',
                     '/politica_privacidade','/como-funciona','/politic','/legal','/Aviso','/noticias','/news',
                     '/perguntasfrequentes']
len(useless_link_list)

65

Iniciando bag of words para item 2

In [13]:
bag_of_word = ['psico','crp','especiali','organizacional','trabalho','trânsito','escolar','educacional','Jurídica','judicial',
              'esporte','clínic','hospitalar','psicopedagogia','psicopedagog','psicomotricidade','sexo','trauma','psiquiátrico',
              'psiquia','patologia','pasciente','consulente','preço','endereço','end','fone','telefone','cel','celular',
              'contato','r$','agend','atendimento','atender','perfil','experiência','humor','extresse','problema','bipolar',
              'orient','qualidade','vida','comportamental','comportamento','comportar','sofrer','sofrimento','terapia','sexo',
              'análise','conhecimento','auto','família','fobia','sexualidade','homo','lgbt','violência','doméstica','abuso',
              'abusiv','mental','saúde','infância','infânti','infanti','medos','depressão','ansiedade','transtorno','suicídio',
              'morte','luto','deficiência','social','consulta','sessão','local','adoção','adotar','filh','filiação','aprend',
              'obesidade','obes','emagrec','bulemia','pânico','síndrome','adolescente','adult','casal','casa','casamento',
              'compulsões','dependencias','drogas','alcool','sono','acolhimento','estado','conflitos','gravidez','maternidade',
              'assertividade','amor','amoros','psicanalista','psicanal','psicanál','psicoterapia','formação','graduação']
len(bag_of_word)

115

Definindo método que reordena a fronteira por links com prefixos semelhantes, ou seja, uma ordem lexicográfica - Item 3.

In [14]:
def orderLinks(links):
    return sorted(links, key=str.lower)

Para o item 4 iremos criar um classificador de links que será treinado e retreinado durante a execução do crawler.

In [15]:
modelNB = MultinomialNB()

In [ ]:

time_to_sleep = 1
limiar = 1000
dict_link_accessed = {}
rate_bow = 1
link_anchor_pos_list = []
link_anchor_neg_list = []
dict_link_to_access_anchor = {}

def putLinkToAccessList(domain, link):
    anchor = link.text
    link = link.get('href')
    if link is None:
        return link_to_access_list
    
    if domain in link:
        if (link not in link_to_access_list):
            link_to_access_list.append(link)
            if link in dict_link_to_access_anchor:
                dict_link_to_access_anchor[link] = dict_link_to_access_anchor[link]+[str(anchor)]
            else:
                dict_link_to_access_anchor[link] = [str(anchor)]
            return link_to_access_list
    elif link.startswith("/"):
        if (domain+link not in link_to_access_list):
            link_to_access_list.append(domain+link)
            if link in dict_link_to_access_anchor:
                dict_link_to_access_anchor[link] = dict_link_to_access_anchor[link]+[str(anchor)]
            else:
                dict_link_to_access_anchor[link] = [str(anchor)]

            return link_to_access_list
     

    
def calcRateBOW(text):
    listRet = []
    for word in bag_of_word:
        if word in text:
            listRet.append(word)
    return listRet
    
for i in range(len(df)):
    domain_block_list = []
    link_accessed_list = []
    link_to_access_list = []
    #iterando na semente
    domain = df['dominio'][i]
    print("Crawleando a semente:\t"+domain)
    link_accessed_list.append(df['semente'][i])    
    path = df['localpath'][i]
    
    if(df['robotstxt'][i]):
        domain_block_list = getDisallows(path)
        print("Esta semente contém um Robots.txt com restrições em:\t"+str(domain_block_list))
        
        
    url = path+df['htmlname'][i]+".html"
    parsed_page = BeautifulSoup(open(url,encoding="latin1"),"html.parser")
    print("Iniciando Crawler nesta semente em:\t"+str(url))
    total_links = parsed_page.find_all('a')
    print("Total de links na página:\t"+str(len(total_links)))

    lenght_front_count_aux = len(link_to_access_list)
    
    
    for link in total_links:
        putLinkToAccessList(domain,link)                           
    link_to_access_list=orderLinks(link_to_access_list)                        
    lenght_front_count_aux = len(link_to_access_list) - lenght_front_count_aux
    print("Total de links adicionados a fronteira:\t"+str(lenght_front_count_aux))
    
    time_to_sleep_aux = random.randint(1,time_to_sleep)
    print("Aguardando 1 segundos para a próxima requisição")
    #time.sleep(time_to_sleep_aux)
    
    count_intern_aux = 0
    for link in link_to_access_list:
        if(not link.startswith("http://") and not link.startswith("https://")):
            link = "http://"+link
        
        block = False
        for blockitem in domain_block_list:
            if(blockitem in link):
                block = True
                print("Requisição falhou, bloqueou por robots.txt")
                break
        
        for uselessItem in useless_link_list:
            if(uselessItem in link):
                block = True
                print("Requisição falhou, bloqueou por link inútil")
                break
        if(not block and link not in link_accessed_list):
            try:
                print("Tentando acessar "+link+"   ...")
                ret=rq.get(link)
                if 'text/html' in ret.headers['Content-Type']:
                    print("Página acessada com sucesso")                    

                    print("Checando taxa de BOW para a página atual")
                    list_bow = calcRateBOW(ret.text)
                    rate_bow_aux = len(list_bow)
                    if rate_bow_aux >= rate_bow:
                        print("A página está dentro da taxa de BOW")

                        link_accessed_list.append(link)

                        if count_intern_aux == 100:
                            if rate_bow<=40:
                                rate_bow=rate_bow+2

                            vectorizer = CountVectorizer(analyzer="word")
                            print(link_anchor_pos_list+list_bow+link_anchor_neg_list)
                            freq_anchors = vectorizer.fit_transform(link_anchor_pos_list+list_bow+link_anchor_neg_list)
                            list_pos = []
                            for elem in link_anchor_pos_list+list_bow:
                                list_pos.append(1)
                            list_neg = []
                            for elem in link_anchor_neg_list:
                                list_neg.append(1)

                            classes = list_pos + list_neg
                            modelNB.fit(freq_anchors,classes)

                            for elem in link_to_access_list:
                                if elem in dict_link_to_access_anchor:
                                    anchors = vectorizer.transform(dict_link_to_access_anchor[elem])
                                    predict = modelNB.predict(anchors)
                                    if predict[0]==0:
                                        link_to_access_list.remove(elem)
                                        useless_link_list.append(elem)

                            #modelNB.predict(link_to_access_list)
                            count_intern_aux = 0

                        if len(link_accessed_list)==limiar:
                            print("LIMIAR ATINGIDO! Foram acessadas "+str(limiar)+" links nesta semente\n\n")
                            print("Crawler concluiu atividade na semente "+domain+"   ...\n")
                            print("\n---------------------\n----\n---------------------\n\n")
                            dict_link_accessed[domain] = link_accessed_list
                            break
                    else:
                        print("A página não contem a taxa limite de BOW.")

                    parsed_page = BeautifulSoup(ret.text,"html.parser")                    

                    print("Capturando links para a fronteira")
                    total_links = parsed_page.find_all('a')
                    print("Total de links na página:\t"+str(len(total_links)))

                    lenght_front_count_aux = len(link_to_access_list)
                    print("OOOI:"+str(rate_bow_aux))
                    for linkItem in total_links:
                        putLinkToAccessList(domain, linkItem)
                        link_to_access_list=orderLinks(link_to_access_list)
                        if rate_bow_aux >= rate_bow:
                            if linkItem.get('text') not in link_anchor_pos_list:
                                if linkItem.get('text') is not None:
                                    link_anchor_pos_list.append(linkItem.get('text'))
                        else:
                            if linkItem.get('text') not in link_anchor_neg_list:
                                if linkItem.get('text') is not None:
                                    print("BBBBBBBBBBBBBBBBBBBBBBB")
                                    link_anchor_neg_list.append(linkItem.get('text'))

                    lenght_front_count_aux = len(link_to_access_list) - lenght_front_count_aux

                    print("Total de links adicionados a fronteira:\t"+str(lenght_front_count_aux))
                    print("Tamanho atual da fronteira:\t"+str(len(link_to_access_list)))
                    print("Crawler concluiu sua passada por "+str(link)+"   ...")
                else:
                    print("O conteúdo do link não é html. O Crawler continuará do próximo na fronteira!")
            except:
                print("Erro ao acessar o link "+str(link)+"\nO Crawler continuará do próximo na fronteira!")
        
        count_intern_aux = count_intern_aux+1
        time_to_sleep_aux = random.randint(1,time_to_sleep)
        print("Aguardando 1 segundos para a próxima requisição")
        #time.sleep(time_to_sleep_aux)
    
        
    dict_link_accessed[domain] = link_accessed_list                
    print("Crawler concluiu atividade na semente "+domain+"   ...\n")
    print("\n---------------------\n----\n---------------------\n\n")

print("Crawler finalizou sua atividade")






Crawleando a semente:	srlupa.com.br
Iniciando Crawler nesta semente em:	pages/sementes/srlupa/srlupa.html
Total de links na página:	72
Total de links adicionados a fronteira:	35
Aguardando 1 segundos para a próxima requisição
Requisição falhou, bloqueou por link inútil
Aguardando 1 segundos para a próxima requisição
Requisição falhou, bloqueou por link inútil
Aguardando 1 segundos para a próxima requisição
Requisição falhou, bloqueou por link inútil
Aguardando 1 segundos para a próxima requisição
Requisição falhou, bloqueou por link inútil
Aguardando 1 segundos para a próxima requisição
Tentando acessar http://srlupa.com.br/   ...
Página acessada com sucesso
Checando taxa de BOW para a página atual
A página está dentro da taxa de BOW
Capturando links para a fronteira
Total de links na página:	72
OOOI:19
Total de links adicionados a fronteira:	0
Tamanho atual da fronteira:	35
Crawler concluiu sua passada por http://srlupa.com.br/   ...
Aguardando 1 segundos para a próxima requisição
Req

Capturando links para a fronteira
Total de links na página:	1569
OOOI:16
Total de links adicionados a fronteira:	325
Tamanho atual da fronteira:	773
Crawler concluiu sua passada por http://srlupa.com.br/profissionais/construcao-e-reformas/eletricista/   ...
Aguardando 1 segundos para a próxima requisição
Tentando acessar http://srlupa.com.br/profissionais/construcao-e-reformas/pedreiro/   ...
Página acessada com sucesso
Checando taxa de BOW para a página atual
A página está dentro da taxa de BOW
Capturando links para a fronteira
Total de links na página:	899
OOOI:14
Total de links adicionados a fronteira:	53
Tamanho atual da fronteira:	826
Crawler concluiu sua passada por http://srlupa.com.br/profissionais/construcao-e-reformas/pedreiro/   ...
Aguardando 1 segundos para a próxima requisição
Tentando acessar http://srlupa.com.br/profissionais/construcao-e-reformas/pintor/   ...
Página acessada com sucesso
Checando taxa de BOW para a página atual
A página está dentro da taxa de BOW
Captu

Página acessada com sucesso
Checando taxa de BOW para a página atual
A página está dentro da taxa de BOW
Capturando links para a fronteira
Total de links na página:	3
OOOI:14
Total de links adicionados a fronteira:	1
Tamanho atual da fronteira:	35
Crawler concluiu sua passada por http://serviceira.com/termos   ...
Aguardando 1 segundos para a próxima requisição
Tentando acessar http://serviceira.com/trabalhos   ...
Página acessada com sucesso
Checando taxa de BOW para a página atual
A página está dentro da taxa de BOW
Capturando links para a fronteira
Total de links na página:	14
OOOI:7
Total de links adicionados a fronteira:	4
Tamanho atual da fronteira:	39
Crawler concluiu sua passada por http://serviceira.com/trabalhos   ...
Aguardando 1 segundos para a próxima requisição
Crawler concluiu atividade na semente serviceira.com   ...


---------------------
----
---------------------


Crawleando a semente:	conectu.com.br
Iniciando Crawler nesta semente em:	pages/sementes/conectu/conect

Página acessada com sucesso
Checando taxa de BOW para a página atual
A página está dentro da taxa de BOW
Capturando links para a fronteira
Total de links na página:	317
OOOI:15
Total de links adicionados a fronteira:	251
Tamanho atual da fronteira:	1663
Crawler concluiu sua passada por https://www.telelistas.net/ba/salvador   ...
Aguardando 1 segundos para a próxima requisição
Tentando acessar https://www.telelistas.net/ba/salvador/clinicas+de+psicologia?gads=no   ...
Página acessada com sucesso
Checando taxa de BOW para a página atual
A página está dentro da taxa de BOW
Capturando links para a fronteira
Total de links na página:	305
OOOI:27
Total de links adicionados a fronteira:	181
Tamanho atual da fronteira:	1844
Crawler concluiu sua passada por https://www.telelistas.net/ba/salvador/clinicas+de+psicologia?gads=no   ...
Aguardando 1 segundos para a próxima requisição
Tentando acessar https://www.telelistas.net/br/cidade/clinicas+psiquiatricas?gads=no   ...
Página acessada com suces

Página acessada com sucesso
Checando taxa de BOW para a página atual
A página está dentro da taxa de BOW
Capturando links para a fronteira
Total de links na página:	73
OOOI:28
Total de links adicionados a fronteira:	1
Tamanho atual da fronteira:	4220
Crawler concluiu sua passada por https://www.telelistas.net/locais/br/cidade/clinicas+de+psicologia/340773225/andre+luiz+braga+queiroz+psicologo+crp+014773   ...
Aguardando 1 segundos para a próxima requisição
Tentando acessar https://www.telelistas.net/locais/br/cidade/clinicas+de+psicologia/340773225/andre+luiz+braga+queiroz+psicologo+crp+014773#contato_email   ...
Página acessada com sucesso
Checando taxa de BOW para a página atual
A página está dentro da taxa de BOW
Capturando links para a fronteira
Total de links na página:	73
OOOI:28
Total de links adicionados a fronteira:	0
Tamanho atual da fronteira:	4220
Crawler concluiu sua passada por https://www.telelistas.net/locais/br/cidade/clinicas+de+psicologia/340773225/andre+luiz+braga+q

...

Página acessada com sucesso
Checando taxa de BOW para a página atual
A página não contem a taxa limite de BOW.
Capturando links para a fronteira
Total de links na página:	8
OOOI:7
Total de links adicionados a fronteira:	6
Tamanho atual da fronteira:	8886
Crawler concluiu sua passada por https://www.telelistas.net/templates/Redireciona.aspx?origem=Facebook&url=https%3A%2F%2Fwww.telelistas.net%2Flocais%2Fsp%2Fsao+paulo%2Fpsicopedagogos%2F335478638%2Fclinica+espaco+construir&local=controle   ...
Aguardando 1 segundos para a próxima requisição
Tentando acessar https://www.telelistas.net/templates/Redireciona.aspx?origem=Facebook&url=https%3A%2F%2Fwww.telelistas.net%2Flocais%2Fsp%2Fsao+paulo%2Fpsicopedagogos%2F340659215%2Fpsicologa+thais&local=controle   ...
Página acessada com sucesso
Checando taxa de BOW para a página atual
A página não contem a taxa limite de BOW.
Capturando links para a fronteira
Total de links na página:	8
OOOI:7
Total de links adicionados a fronteira:	6
Tamanho atual 

Página acessada com sucesso
Checando taxa de BOW para a página atual
A página não contem a taxa limite de BOW.
Capturando links para a fronteira
Total de links na página:	72
OOOI:12
Total de links adicionados a fronteira:	0
Tamanho atual da fronteira:	8896
Crawler concluiu sua passada por https://www.telelistas.net/templates/Redireciona.aspx?origem=Twitter&url=https://www.telelistas.net/locais/pr/curitiba/psicopedagogos/336836095/susane+volker+agostinho+psicopedagoga&local=controle&nome=Susane Völker Agostinho - Psicopedagoga&cidade=PR   ...
Aguardando 1 segundos para a próxima requisição
Tentando acessar https://www.telelistas.net/templates/Redireciona.aspx?origem=Twitter&url=https://www.telelistas.net/locais/pr/maringa/psicopedagogos/335882397/educahcom+assessoria+pedagogica&local=controle&nome=Educah.Com Assessoria Pedagógica&cidade=PR   ...
Página acessada com sucesso
Checando taxa de BOW para a página atual
A página não contem a taxa limite de BOW.
Capturando links para a fronteir

Página acessada com sucesso
Checando taxa de BOW para a página atual
A página não contem a taxa limite de BOW.
Capturando links para a fronteira
Total de links na página:	72
OOOI:12
Total de links adicionados a fronteira:	0
Tamanho atual da fronteira:	8896
Crawler concluiu sua passada por https://www.telelistas.net/templates/Redireciona.aspx?origem=Twitter&url=https://www.telelistas.net/locais/sc/florianopolis/psicopedagogos/300794930/consultorio+psicopedagogico+ressignificar&local=controle&nome=Consultório Psicopedagógico Ressignificar&cidade=SC   ...
Aguardando 1 segundos para a próxima requisição
Tentando acessar https://www.telelistas.net/templates/Redireciona.aspx?origem=Twitter&url=https://www.telelistas.net/locais/sc/itajai/psicopedagogos/300806993/caroline+tomio+psicopedagoga&local=controle&nome=Caroline Tomio - Psicopedagoga&cidade=SC   ...
Página acessada com sucesso
Checando taxa de BOW para a página atual
A página não contem a taxa limite de BOW.
Capturando links para a fro

Página acessada com sucesso
Checando taxa de BOW para a página atual
A página não contem a taxa limite de BOW.
Capturando links para a fronteira
Total de links na página:	0
OOOI:0
Total de links adicionados a fronteira:	0
Tamanho atual da fronteira:	9186
Crawler concluiu sua passada por http://telelistas.net//www.ophertas.net/belo+horizonte-mg   ...
Aguardando 1 segundos para a próxima requisição
Tentando acessar http://telelistas.net//www.ophertas.net/brasilia-df   ...
Página acessada com sucesso
Checando taxa de BOW para a página atual
A página não contem a taxa limite de BOW.
Capturando links para a fronteira
Total de links na página:	0
OOOI:0
Total de links adicionados a fronteira:	0
Tamanho atual da fronteira:	9186
Crawler concluiu sua passada por http://telelistas.net//www.ophertas.net/brasilia-df   ...
Aguardando 1 segundos para a próxima requisição
Tentando acessar http://telelistas.net//www.ophertas.net/curitiba-pr   ...
Página acessada com sucesso
Checando taxa de BOW para a 

Página acessada com sucesso
Checando taxa de BOW para a página atual
A página não contem a taxa limite de BOW.
Capturando links para a fronteira
Total de links na página:	0
OOOI:0
Total de links adicionados a fronteira:	0
Tamanho atual da fronteira:	9186
Crawler concluiu sua passada por http://telelistas.net/br/psicopedagogos?letra=H&randsort=805634560   ...
Aguardando 1 segundos para a próxima requisição
Tentando acessar http://telelistas.net/br/psicopedagogos?letra=I&randsort=805634560   ...
Página acessada com sucesso
Checando taxa de BOW para a página atual
A página não contem a taxa limite de BOW.
Capturando links para a fronteira
Total de links na página:	0
OOOI:0
Total de links adicionados a fronteira:	0
Tamanho atual da fronteira:	9186
Crawler concluiu sua passada por http://telelistas.net/br/psicopedagogos?letra=I&randsort=805634560   ...
Aguardando 1 segundos para a próxima requisição
Tentando acessar http://telelistas.net/br/psicopedagogos?letra=J&randsort=805634560   ...
Pá